In [35]:
%tensorflow_version 1.x
import tensorflow as tf

tf.set_random_seed(777)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
filename_queue = tf.train.string_input_producer(['/content/drive/My Drive/DeepLearningZeroToAll-master/data-03-diabetes.csv'], shuffle=False, name='filname_queue')
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

In [37]:
record_defaults = [[0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

In [38]:
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

In [39]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [40]:
# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(-tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

In [41]:
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

In [42]:
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

In [43]:
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [44]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [45]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [46]:
for step in range(10001):
  x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
  cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
  if step % 200 == 0:
      print(step, "Cost: ", cost_val, "\nPrediction\n", hy_val)

# Accuracy report
h, c, a = sess.run([hypothesis, hypothesis, accuracy], feed_dict={X: x_batch, Y: y_batch})
print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)

0 Cost:  0.6043873 
Prediction
 [[0.41456637]
 [0.46052617]
 [0.5848103 ]
 [0.4024758 ]
 [0.13653824]
 [0.6344215 ]
 [0.5236409 ]
 [0.7672589 ]
 [0.32675472]
 [0.6093476 ]]
200 Cost:  0.5652961 
Prediction
 [[0.5117448 ]
 [0.5082553 ]
 [0.5619871 ]
 [0.6886151 ]
 [0.3851818 ]
 [0.6876467 ]
 [0.5420646 ]
 [0.72186816]
 [0.6127306 ]
 [0.6562671 ]]
400 Cost:  0.8531623 
Prediction
 [[0.5328662 ]
 [0.5517708 ]
 [0.6055845 ]
 [0.75975466]
 [0.6745447 ]
 [0.5978962 ]
 [0.71757925]
 [0.80792075]
 [0.6513512 ]
 [0.6653937 ]]
600 Cost:  0.5498949 
Prediction
 [[0.67006   ]
 [0.8776516 ]
 [0.6631582 ]
 [0.41606432]
 [0.6117845 ]
 [0.6506208 ]
 [0.2615405 ]
 [0.49866894]
 [0.8280969 ]
 [0.7855722 ]]
800 Cost:  0.6886107 
Prediction
 [[0.5741959 ]
 [0.5044658 ]
 [0.7293858 ]
 [0.7708098 ]
 [0.47165623]
 [0.6591931 ]
 [0.6349035 ]
 [0.5799083 ]
 [0.58568376]
 [0.54702175]]
1000 Cost:  0.43493056 
Prediction
 [[0.7449668 ]
 [0.7405709 ]
 [0.86458266]
 [0.67428887]
 [0.77595365]
 [0.646433  ]
 [0.674

In [47]:
coord.request_stop()
coord.join(threads)